In [1]:
import sys
import time
import numpy as np
import PIL.Image
from io import BytesIO
import tensorflow as tf
import boto3
import cv2
import IPython.display
import pytesseract
import os
import re
import pprint
import pandas as pd
import fuzzywuzzy
import string
import random
from fuzzywuzzy import process
import chardet
import urllib.request
import ipywidgets as widgets
from IPython.display import Image
from IPython.core.display import HTML 

/usr/local/lib/python3.5/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
"""
Generating an artificial NADRA database.
"""
nadra_df = pd.DataFrame({'Name': ['AAAA',
                                  'BBBB', 
                                  'CCCC', 
                                  'DDDD', 
                                  'EEEE'], 
                         'Father Name': ['FFFF',
                                         'GGGG', 
                                         'HHHH', 
                                         'IIII', 
                                         'JJJJ'],
                         'Gender': ['M',
                                    'M', 
                                    'M', 
                                    'M', 
                                    'M'],
                         'Country of Stay': ['Pakistan',
                                             'Pakistan', 
                                             'Pakistan', 
                                             'China, Mainland', 
                                             'Pakistan'],
                         'CNIC': ['11111-1111111-1',
                                  '22222-2222222-2', 
                                  '33333-3333333-3', 
                                  '44444-4444444-4', 
                                  '55555-5555555-5'],
                         'Date of Birth': ['23.03.1995', 
                                           '26.06.1995', 
                                           '10.07.1996', 
                                           '09.10.1972', 
                                           '12.12.1963'],
                         'Date of Issue': ['25.05.2016', 
                                           '25.06.2014', 
                                           '26.11.2015', 
                                           '24.04.2017', 
                                           '09.09.2014'],
                         'Date of Expiry': ['25.05.2026', 
                                            '25.06.2024', 
                                            '26.11.2025', 
                                            '24.04.2027', 
                                            '09.09.2021'],
                         'Cell Number': ['+923333333333', 
                                         '+923134444444', 
                                         '+923215555555', 
                                         '+923136666666', 
                                         '+923007777777']
                        })

"""
Making CNIC the index of the dataset file
"""
nadra_df.index = nadra_df.CNIC

In [3]:
nadra_df

,CNIC,Cell Number,Country of Stay,Date of Birth,Date of Expiry,Date of Issue,Father Name,Gender,Name
CNIC,,,,,,,,,
11111-1111111-1,11111-1111111-1,+923333333333,Pakistan,23.03.1995,25.05.2026,25.05.2016,FFFF,M,AAAA
22222-2222222-2,22222-2222222-2,+923134444444,Pakistan,26.06.1995,25.06.2024,25.06.2014,GGGG,M,BBBB
33333-3333333-3,33333-3333333-3,+923215555555,Pakistan,10.07.1996,26.11.2025,26.11.2015,HHHH,M,CCCC
44444-4444444-4,44444-4444444-4,+923136666666,"China, Mainland",09.10.1972,24.04.2027,24.04.2017,IIII,M,DDDD
55555-5555555-5,55555-5555555-5,+923007777777,Pakistan,12.12.1963,09.09.2021,09.09.2014,JJJJ,M,EEEE


In [5]:
def validate_from_nadra(record):
    valid = True
    unequal_fields = []
    doc_values = []
    nadra_values = []
    for k, v in doc_dict.items():
#         print(k, v, record[k])
        if v != record[k]:
#             print(v)
            unequal_fields.append(k)
            doc_values.append(doc_dict[k])
            nadra_values.append(record[k])
            valid = False
    return valid, unequal_fields, doc_values, nadra_values

In [6]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(string_list, string_to_match, min_ratio = 49):
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(choices=string_list, query=string_to_match, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
#     print(close_matches)

#     # get the rows of all the close matches in our dataframe
#     rows_with_matches = df[column].isin(close_matches)

#     # replace all rows with close matches with the input matches 
#     df.loc[rows_with_matches, column] = string_to_match
    return close_matches

In [7]:
def reorder_dates(l):
    """
    Input: Takes a list with three different dot-separated dates.
    For example: ['23.04.1997','12.11.2021', '22.05.2012']
    Output: Return list of dates in sorted manner based on the year in an ascending order.
    For example: ['23.04.1997', '22.05.2012', '12.11.2021']
    """
    date1 = l[0]
    date2 = l[1]
    date3 = l[2]
    
    date1 = date1.split('.')
    date2 = date2.split('.')
    date3 = date3.split('.')
    dates_list = [date1, date2, date3]
    srtd = sorted(dates_list,key=lambda l:l[2], reverse=False)
    for i in range(len(srtd)):
        srtd[i] = '.'.join(srtd[i])
    return srtd

In [8]:
def id_generator(size=6, chars=string.ascii_lowercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

In [9]:
def validate_images(byteSelfie, byteCard):
    validated = False
    client=boto3.client('rekognition', aws_access_key_id="XXXXXXXXXXXXXXXXXX",
                        aws_secret_access_key="yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy",)

    response=client.compare_faces(SimilarityThreshold=40,
                                      SourceImage={'Bytes': byteSelfie},
                                      TargetImage={'Bytes': byteCard})

    for faceMatch in response['FaceMatches']:
        validated = True
        position = faceMatch['Face']['BoundingBox']
        confidence = str(faceMatch['Face']['Confidence'])
#         print('The face at ' +
#                    str(position['Left']) + ' ' +
#                    str(position['Top']) +
#                    ' matches with ' + confidence + '% confidence')
    return validated

In [10]:
def read_card(byteCard):
    client=boto3.client('rekognition')
    response = client.detect_text(Image={'Bytes': byteCard})

    # print(response)
    textDetections=response['TextDetections']
#     print(response)
#     print('Matching faces')
    text_list = []
    for text in textDetections:
            text_list.append(text['DetectedText'])
            # print('Confidence: ' + "{:.2f}".format(text['Confidence']) + "%")
            # print('Id: {}'.format(text['Id']))
            # if 'ParentId' in text:
            #     print('Parent Id: {}'.format(text['ParentId']))
            # print('Type:' + text['Type'])
    return text_list

In [11]:
def one_time_password(num):
    
    # Create an SNS client
    client=boto3.client('sns', aws_access_key_id="XXXXXXX",
                        aws_secret_access_key="yyyyyyyyyyyyyyyyyyyy", 
                        region_name="us-west-2",)
    otp = id_generator()
    # Send your sms message.
    client.publish(
        PhoneNumber=num,
        Message=otp
    )
    return otp

In [ ]:
# Selfie
"""
IP Webcam App from here: https://play.google.com/store/apps/details?id=com.pas.webcam&hl=en
is required, in order to set up a server at 192.168.20.181:8000 to fetch live images from the
cell phone's camera.
"""
url = "http://192.168.10.10:8080/shot.jpg"
byteSelfie = urllib.request.urlopen(url).read()
imgResp=urllib.request.urlopen(url)
# print("imgResp: ", imgResp)
# print("imgResp: {}".format(imgResp))
selfieNp=np.array(bytearray(imgResp.read()),dtype=np.uint8)
# print("imgNp: {} {}".format(imgNp, imgNp.shape))
# print("reshaped: {}".format(imgNp.reshape(300, 400 ,3)))
selfie=cv2.imdecode(selfieNp,-1)
selfie=cv2.cvtColor(selfie, cv2.COLOR_BGR2RGB)
def showarray(a, fmt='jpeg'):
    a = np.uint8(a)
    f = BytesIO()
    PIL.Image.fromarray(a, mode="RGB").save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))
showarray(selfie)

In [ ]:
# Card
url = "http://192.168.10.10:8080/shot.jpg"
byteCard = urllib.request.urlopen(url).read()
imgResp=urllib.request.urlopen(url)
# print("imgResp: {}".format(imgResp))
cardNp=np.array(bytearray(imgResp.read()),dtype=np.uint8)
# print("imgNp: {} {}".format(imgNp, imgNp.shape))
# print("reshaped: {}".format(imgNp.reshape(300, 400 ,3)))
# card = cv2.imread('cnics/abdul.jpg')
card=cv2.imdecode(cardNp,-1)
card=cv2.cvtColor(card, cv2.COLOR_BGR2RGB)
def showarray(a, fmt='jpeg'):
    a = np.uint8(a)
    f = BytesIO()
    PIL.Image.fromarray(a, mode="RGB").save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))
showarray(card)

In [19]:
forward = True
if validate_images(byteSelfie, byteCard):
    print("Selfie matches with the document photo. Proceeding...")
    forward = True
else:
    print("Selfie doesn't match with the document\nCan't proceed any further")
    forward = False

Selfie matches with the document photo. Proceeding...


In [27]:
if not forward:
    print("Can't proceed without solving the previous issue: Incompatible selfie and document picture")
else:
    t = read_card(byteCard)

    strings = [
        'Name', 'Father Name', 'Gender Country of Stay'
    ]
    misspelled_strings = [
        'Nane', 'Nama', 'Mame', 'NamE', 'Father Nane', 'Futher Mame', 'Gendar Country Stay', 'Gender Country of',
        'Father', 'ather', 'ather Nam', 'Fathe Name'
    ]

    try:
        for i, x in enumerate(strings):
            wrong_list = replace_matches_in_column(t, strings[i])
    #         print(wrong_list)
    #         print(wrong_list[0])
    #         print(t.index(wrong_list[0]))
            t[t.index(wrong_list[0])] = strings[i]
        forward = True
    except:
        forward = False
        print("Scanned document is unclear! Please scan again with a better image quality.")

In [ ]:
if not forward:
    print("Can't proceed without solving previous issue: Scanned document is unclear. Please scan again.")
else:
    try:
        name = t[t.index('Name') + 1].title()
        father_name = t[t.index('Father Name') + 1].title()
        gender_country = t.index('Gender Country of Stay')
        gender = t[gender_country + 1]
        country = t[gender_country + 1]
        if len(gender) > 1:
            if len(gender.split(' ')) > 1:
                country = gender.split(' ', 1)[1]
        else:
            country = t[gender_country + 2]
        if len(country) < 2:
            country = t[gender_country + 1]

        re_cnic = re.compile("^[0-9+]{5}-[0-9+]{7}-[0-9]{1}$")
        re_gender = re.compile("^M$|^F$")
    except:
        print("Either the scanned card is deformed, invalid, or unclear. Please try again with a clearer picture with no\n missing data.")
    try:
        gender = list(filter(re_gender.match, t))[0]
        forward = True
    except:
        if len(gender) > 1:
            gender = gender[0]
    try:
        cnic = list(filter(re_cnic.match, t))[0] # Read Note
        re_dates = re.compile("^[0-9+]{2}.[0-9+]{2}.[0-9]{4}$")
        date_birth, date_issue, date_expiry = reorder_dates(list(set(list(filter(re_dates.match, t)))))
        doc_dict = {
            'Name': name,
            'Father Name': father_name,
            'Gender': gender,
            'Country of Stay': country,
            'CNIC': cnic,
            'Date of Birth': date_birth,
            'Date of Issue': date_issue,
            'Date of Expiry': date_expiry
        }

    # print("Name: {}\nFather Name: {}\nGender: {}\nCountry of Stay: {}\
    #       \nCNIC: {}\nDate of Birth: {}\nDate Issue: {}\nDate Expiry: {}"\
    #       .format(name, father_name, gender, country, cnic, date_birth, date_issue, date_expiry))
        pprint.pprint(doc_dict)
        try:
            record = nadra_df.loc[doc_dict['CNIC']]
            record = record.to_dict()
            forward = True
        except:
            forward = False
            print("No record found for the CNIC {} in the NADRA database.".format(doc_dict['CNIC']))
    except:
        print("CNIC ID or Dates are either unclear, or removed. Please scan with a clearer photo, or use a card which isn't damaged.")

In [ ]:
if not forward:
    print("Can't proceed without resolving previous issue: NADRA record not found against the CNIC {}\n or scanned document is unclear.".format(doc_dict['CNIC']))
else:
    v, u, d, n = validate_from_nadra(record)
    if v==False:
        forward = False
        print("\tDifference between scanned document and NADRA record found.\n \
        If you believe that there isn't any difference, then please scan your card again\n \
        with a better quality photo.")
        print("Following differences were found:")
        for a, b, c in list(zip(u, d, n)):
            print("Field: {}\tDocument Value: {}\tNADRA Value: {}".format(a, b, c))
    else:
        forward = True
        print("NADRA verification complete!\nPlease enter an OTP which was sent to you\n\
        on your cell number: {}".format(record['Cell Number']))
        otp = one_time_password(record['Cell Number'])

In [31]:
if not forward:
    print("Can't proceed without solving previous issue: Document data not according to NADRA.")
else:
    otp_inp = str(input("Enter the OTP received: "))
    if otp == otp_inp:
        print("Registeration Successful!")
    else:
        print("OTP failed! Please enter the correct OTP")

Enter the OTP received: 59ybxt
Registeration Successful!


In [37]:
otp

'kjq4w8'